# Analysis Sample

In [1]:
from pathlib import Path
import PIL
import numpy as np
import matplotlib.pyplot as plt 

In [2]:
# Cancer pathology samples
data_dir = "./data/"
img_paths = {
    path.stem: path for path in Path(data_dir).glob("*.TIF")
}

print(img_paths)

images = {
    name: np.array(PIL.Image.open(path))
    for name, path in img_paths.items()
}

{'Dual fusion': WindowsPath('data/Dual fusion.TIF'), 'separation probe': WindowsPath('data/separation probe.TIF'), 'Trisomy 8': WindowsPath('data/Trisomy 8.TIF')}


In [3]:
from ufish.api import UFish
from cellpose import models

# initialize cellpose model
cp = models.Cellpose(gpu=False, model_type="nuclei")

# initialize ufish model
uf = UFish()
uf.load_weights("./v1.0.1-DNAFISH_model.onnx")
# weights link: https://huggingface.co/GangCaoLab/U-FISH/blob/main/finetune_models/v1.0.1-DNAFISH_model.pth

from utils import pipeline, plot_figs, plot_all_rois

2024-05-24 12:54:21.529 | INFO     | ufish.api:_load_onnx:292 - Loading ONNX from ./v1.0.1-DNAFISH_model.onnx


In [4]:
from shutil import rmtree
import os
main_dir = "results"
os.makedirs(main_dir, exist_ok=True)
for name, img in images.items():
    print(name)
    res_dir = f"./{main_dir}/{name}"
    if os.path.exists(res_dir):
        rmtree(f"./{res_dir}")
    os.mkdir(f"./{res_dir}")

    if name.startswith("Trisomy"):
        signal_channels=[0]
    else:
        signal_channels=[0, 1]
    table, rois, masks, signals= pipeline(cp, uf, img, signal_channels=signal_channels)

    plot_figs(rois, masks, signals, res_dir)

    fig = plot_all_rois(rois, masks, signals, colors={"ch1": "hotpink", "ch2": "lime", "ch1+ch2": "yellow"})
    fig.savefig(f"{res_dir}/{name}_all_cells.pdf")
    plt.close(fig)

    table.to_csv(f"{res_dir}/{name}_results.csv", index=False)

Dual fusion
Processing image:
Image shape: (3000, 4096, 3)
Step 1: segment cells


2024-05-24 12:56:05.498 | INFO     | ufish.api:predict:464 - Axes not specified, infering from image shape.
2024-05-24 12:56:05.499 | INFO     | ufish.api:predict:466 - Infered axes: yx, image shape: (128, 128)


Step 2: extract ROIs
Step 3: call spots and assign spots


2024-05-24 12:56:05.630 | INFO     | ufish.api:predict:464 - Axes not specified, infering from image shape.
2024-05-24 12:56:05.631 | INFO     | ufish.api:predict:466 - Infered axes: yx, image shape: (128, 128)
2024-05-24 12:56:05.660 | INFO     | ufish.api:predict:464 - Axes not specified, infering from image shape.
2024-05-24 12:56:05.661 | INFO     | ufish.api:predict:466 - Infered axes: yx, image shape: (128, 128)
2024-05-24 12:56:05.674 | INFO     | ufish.api:predict:464 - Axes not specified, infering from image shape.
2024-05-24 12:56:05.676 | INFO     | ufish.api:predict:466 - Infered axes: yx, image shape: (128, 128)
2024-05-24 12:56:05.704 | INFO     | ufish.api:predict:464 - Axes not specified, infering from image shape.
2024-05-24 12:56:05.706 | INFO     | ufish.api:predict:466 - Infered axes: yx, image shape: (128, 128)
2024-05-24 12:56:05.719 | INFO     | ufish.api:predict:464 - Axes not specified, infering from image shape.
2024-05-24 12:56:05.721 | INFO     | ufish.api:p

separation probe
Processing image:
Image shape: (3000, 4096, 3)
Step 1: segment cells


...

2024-05-24 12:57:48.330 | INFO     | ufish.api:predict:464 - Axes not specified, infering from image shape.
2024-05-24 12:57:48.332 | INFO     | ufish.api:predict:466 - Infered axes: yx, image shape: (128, 128)
2024-05-24 12:57:48.350 | INFO     | ufish.api:predict:464 - Axes not specified, infering from image shape.
2024-05-24 12:57:48.352 | INFO     | ufish.api:predict:466 - Infered axes: yx, image shape: (128, 128)
2024-05-24 12:57:48.391 | INFO     | ufish.api:predict:464 - Axes not specified, infering from image shape.
2024-05-24 12:57:48.394 | INFO     | ufish.api:predict:466 - Infered axes: yx, image shape: (128, 128)
2024-05-24 12:57:48.411 | INFO     | ufish.api:predict:464 - Axes not specified, infering from image shape.
2024-05-24 12:57:48.413 | INFO     | ufish.api:predict:466 - Infered axes: yx, image shape: (128, 128)
2024-05-24 12:57:48.451 | INFO     | ufish.api:predict:464 - Axes not specified, infering from image shape.
2024-05-24 12:57:48.452 | INFO     | ufish.api:p

Trisomy 8
Processing image:
Image shape: (3000, 4096, 3)
Step 1: segment cells


2024-05-24 12:59:16.095 | INFO     | ufish.api:predict:464 - Axes not specified, infering from image shape.
2024-05-24 12:59:16.096 | INFO     | ufish.api:predict:466 - Infered axes: yx, image shape: (128, 128)
2024-05-24 12:59:16.113 | INFO     | ufish.api:predict:464 - Axes not specified, infering from image shape.
2024-05-24 12:59:16.115 | INFO     | ufish.api:predict:466 - Infered axes: yx, image shape: (128, 128)


Step 2: extract ROIs
Step 3: call spots and assign spots


2024-05-24 12:59:16.133 | INFO     | ufish.api:predict:464 - Axes not specified, infering from image shape.
2024-05-24 12:59:16.134 | INFO     | ufish.api:predict:466 - Infered axes: yx, image shape: (128, 128)
2024-05-24 12:59:16.157 | INFO     | ufish.api:predict:464 - Axes not specified, infering from image shape.
2024-05-24 12:59:16.159 | INFO     | ufish.api:predict:466 - Infered axes: yx, image shape: (128, 128)
2024-05-24 12:59:16.180 | INFO     | ufish.api:predict:464 - Axes not specified, infering from image shape.
2024-05-24 12:59:16.182 | INFO     | ufish.api:predict:466 - Infered axes: yx, image shape: (128, 128)
2024-05-24 12:59:16.205 | INFO     | ufish.api:predict:464 - Axes not specified, infering from image shape.
2024-05-24 12:59:16.207 | INFO     | ufish.api:predict:466 - Infered axes: yx, image shape: (128, 128)
2024-05-24 12:59:16.230 | INFO     | ufish.api:predict:464 - Axes not specified, infering from image shape.
2024-05-24 12:59:16.232 | INFO     | ufish.api:p